# word2vec implementation

* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* [word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/abs/1402.3722)

---
# Setups

In [1]:
import cProfile
import sys
import os
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

# Jupyter notebook setups

Auto reolaod causes an error in Jupyter notebooks. Restart the Jupyter kernel for the error:
```TypeError: super(type, obj): obj must be an instance or subtype of type```
See
- https://stackoverflow.com/a/52927102/4281353
- http://thomas-cokelaer.info/blog/2011/09/382/

> The problem resides in the mechanism of reloading modules.
> Reloading a module often changes the internal object in memory which
> makes the isinstance test of super return False.

In [2]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [3]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio
import function.text as text

---
# Data Types


In [4]:
from common.constant import (
    TYPE_INT,
    TYPE_FLOAT,
    TYPE_LABEL,
    TYPE_TENSOR,
)

# Constants

In [5]:
USE_PTB = True
DEBUG = False
VALIDATION = True

TARGET_SIZE = 1   # Size of the target event (word)
CONTEXT_SIZE = 10  # Size of the context.
WINDOW_SIZE = TARGET_SIZE + CONTEXT_SIZE
SAMPLE_SIZE = 5   # Size of the negative samples
VECTOR_SIZE = 100  # Number of features in the event vector.

---

# Data
## Corpus

In [6]:
corpus = "To be, or not to be, that is the question that matters"
_file = "ptb.train.txt"
if USE_PTB:
    if not fileio.Function.is_file(f"~/.keras/datasets/{_file}"):
        path_to_ptb = tf.keras.utils.get_file(
            _file, 
            f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/{_file}'
        )
    corpus = fileio.Function.read_file(path_to_ptb)

In [7]:
examples = corpus.split('\n')[:5]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 
 pierre <unk> N years old will join the board as a nonexecutive director nov. N 
 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 
 rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate 
 a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported 


---
# Event (word) indexing
Index the events that have occurred in the event sequence.

In [8]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing, 
)

In [9]:
word_indexing = EventIndexing(
    name="word_indexing_on_ptb",
    corpus=corpus
)
del corpus

## EventIndexing  for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [10]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'aer' 'banknote' 'berlitz' 'calloway' 'centrust' 'cluett' 'fromstein' 'gitano']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('aer', 2)
('banknote', 3)
('berlitz', 4)
('calloway', 5)
('centrust', 6)
('cluett', 7)
('fromstein', 8)
('gitano', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
aer                  : 5.34860e-06
banknote             : 5.34860e-06
berlitz              : 5.34860e-06
calloway             : 5.34860e-06
centrust             : 5.34860e-06
cluett               : 5.34860e-06
fromstein            : 5.34860e-06
gitano               : 5.34860e-06


## Sampling using the probability

Sample events according to their probabilities.

In [11]:
sample = word_indexing.sample(size=5)
print(sample)

['permanent' 'audience' 'predict' 'f' 'productions']


## Negative Sampling
Sample events not including those events already sampled.

In [12]:
negative_indices = word_indexing.negative_sample_indices(
    size=5, excludes=word_indexing.list_indices(sample)
)
print(f"negative_indices={negative_indices} \nevents={word_indexing.list_events(negative_indices)}")


negative_indices=[8642, 8479, 6278, 2343, 5385] 
events=['doubling' 'gang' 'puerto' 'calls' 'guard']


## Sentence to Sequence

In [13]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
with even brief exposures to it causing symptoms that show up decades later researchers said
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :    34
asbestos        :    63
fiber           :    86
<unk>           :     1
is              :    42
unusually       :    87
<unk>           :     1
once            :    64
it              :    80
enters          :    88
the             :    34
<unk>           :     1

with           :     0
even            :     0
brief           :     0


---
# EventContext

EventContext layer generates ```(event, context)``` pairs from sequences.

In [14]:
%autoreload 2
from layer.preprocessing import (
    EventContext
)

In [15]:
event_context = EventContext(
    name="ev",
    window_size=WINDOW_SIZE,
    event_size=TARGET_SIZE
)

## Context of an event (word) in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [16]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold fields plc was named a nonexecutive director
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[37, 62, 44, 63, 64, 65, 66, 67, 68, 69, 70,  0,  0,  0,  0,  0],
       [29, 30, 31, 50, 51, 43, 44, 52, 53, 54, 55, 56, 57, 37, 38, 39]])

## (event, context) pairs

For each word (event) in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

### Format of the (event, context) pairs

* **E** is the target event indices
* **C** is the context indices

<img src="image/event_context_format.png" align="left"/>

In [17]:
event_context_pairs = event_context.function(sequence)
print(
    f"Event context pairs. Shape %s, Target event size %s, Window size %s." % 
    (event_context_pairs.shape, event_context.event_size, event_context.window_size)
)
event_context_pairs[:10]

Event context pairs. Shape (21, 5), Target event size 1, Window size 5.


array([[44, 37, 62, 63, 64],
       [63, 62, 44, 64, 65],
       [64, 44, 63, 65, 66],
       [65, 63, 64, 66, 67],
       [66, 64, 65, 67, 68],
       [67, 65, 66, 68, 69],
       [68, 66, 67, 69, 70],
       [69, 67, 68, 70,  0],
       [70, 68, 69,  0,  0],
       [31, 29, 30, 50, 51]], dtype=int32)

### (event, context) pairs in textual words

In [18]:
word_indexing.sequence_to_sentence(event_context_pairs[:10])

[['of', 'a', 'form', 'asbestos', 'once'],
 ['asbestos', 'form', 'of', 'once', 'used'],
 ['once', 'of', 'asbestos', 'used', 'to'],
 ['used', 'asbestos', 'once', 'to', 'make'],
 ['to', 'once', 'used', 'make', 'kent'],
 ['make', 'used', 'to', 'kent', 'cigarette'],
 ['kent', 'to', 'make', 'cigarette', 'filters'],
 ['cigarette', 'make', 'kent', 'filters', '<nil>'],
 ['filters', 'kent', 'cigarette', '<nil>', '<nil>'],
 ['old', 'n', 'years', 'and', 'former']]

---
# Word Embedding

Embedding is to train the model to group similar events in a close proximity in the event vector space. If two events e.g. 'pencil' and 'pen' are similar concepts, then their event vectors resides in a close distance in the event space. 

* [Thought Vectors](https://wiki.pathmind.com/thought-vectors)

## Training process

1. Calculate ```Bc```, the BoW (Bag of Words) from context event vectors.
2. Calculate ```Be```,  the BoW (Bag of Words) from target event vectors.
3. The dot product ```Ye = dot(Bc, Be)``` is given the label 1 to get them closer.
4. For each negative sample ```Ws(s)```, the dot product with ```Ys = dot(Be, Ws(s)``` is given the label 0 to get them apart. 
5. ```np.c_[Ye, Ys]``` is fowarded to the logistic log loss layer.

<img src="image/word2vec_backprop_Be.png" align="left"/>


In [19]:
%autoreload 2
from layer import (
    Embedding
)
from optimizer import (
    SGD
)

In [20]:
embedding: Embedding = Embedding(
    name="embedding",
    num_nodes=WINDOW_SIZE,
    target_size=TARGET_SIZE,
    context_size=CONTEXT_SIZE,
    negative_sample_size=SAMPLE_SIZE,
    event_vector_size=VECTOR_SIZE,
    optimizer=SGD(lr=TYPE_FLOAT(0.3)),
    dictionary=word_indexing
)

---
# Word2vec Network

## Construct a sequential network

$ \text {Sentences} \rightarrow EventIndexing \rightarrow EventContext \rightarrow  Embedding \rightarrow Adapter \rightarrow LogisticLogLoss$

In [21]:
#STATE_FILE_20 = "../models/word2vec_vecsize_20.pkl"
STATE_FILE_50 = "../models/word2vec_vecsize_50.pkl"
STATE_FILE_100 = "../models/word2vec_vecsize_100.pkl"

---
# Evaluate the vector space

Verify if the trained model, or the vector space W, has encoded the words in a way that **similar** words are close in the vector space.

* [How to measure the similarity among vectors](https://math.stackexchange.com/questions/4132458)

In [81]:
n = 10
context = "terrorism".split()
word_indices = np.array(word_indexing.list_indices(context), dtype=TYPE_INT)

print(f"Words {context}")
print(f"Word indices {word_indices}")

Words ['terrorism']
Word indices [7228]


## Vector size 50

In [82]:
state = embedding.load(STATE_FILE_50)

fmt="""Model loaded.
event_size %s
context_size: %s
event_vector_size: %s
"""
print(fmt % (
    state["target_size"], 
    state["context_size"], 
    state["event_vector_size"]
))

print(word_indexing.list_events([embedding.predict(word_indices, n)]))

Model loaded.
event_size 1
context_size: 4
event_vector_size: 50

[['colorado' 'observers' 'warrants' 'intended' 'chain' 'hostile' 'drawn' 'newspapers' 'insurers' 'gop']
 ['colorado' 'ivy' 'observers' 'viability' 'diaper' 'drawn' 'rubbermaid' 'warrants' 'priorities' 'raiders']]


## Vector size 100

In [83]:
state = embedding.load(STATE_FILE_100)

fmt="""Model loaded.
event_size %s
context_size: %s
event_vector_size: %s
"""
print(fmt % (
    state["target_size"], 
    state["context_size"], 
    state["event_vector_size"]
))

print(word_indexing.list_events([embedding.predict(word_indices, n)]))

Model loaded.
event_size 1
context_size: 10
event_vector_size: 100

[['crash' 'children' 'morning' 'following' 'tax' 'cold' 'lead' 'share' 'home' 'p']
 ['grumman' 'watching' 'challenges' 'container' 'fired' 'slashing' 'ward' 'decades' 'bates' 'lawrence']]
